In [ ]:
# Grid search
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
param_grid = [
    {
        "min_samples_split": [50, 100],
        "max_depth": [20, 30, 40],
        "n_estimators": [100, 200]
    }
]

# Let's also try some CV

from sklearn.experimental import enable_halving_search_cv # noqa
# now you can import normally from model_selection
from sklearn.model_selection import HalvingGridSearchCV
X_train, X_test, y_train, y_test = train_test_split(df_encoded[features], df_encoded[target], train_size = .9)
from sklearn.model_selection import RepeatedStratifiedKFold
cv = RepeatedStratifiedKFold(n_splits=3, n_repeats=3, random_state=42)

search = HalvingGridSearchCV(
    estimator=RandomForestClassifier(),
    param_grid=param_grid,
    scoring='precision',
    cv=cv,
    n_jobs=-1,
    verbose=1
)

search.fit(X_train, y_train)

# X_train, X_test, y_train, y_test = train_test_split(df_encoded[features], df_encoded[target], train_size = .9)
# from sklearn.model_selection import RepeatedStratifiedKFold
# cv = RepeatedStratifiedKFold(n_splits=3, n_repeats=3)

# search = GridSearchCV(estimator=RandomForestClassifier(), 
#                       param_grid=param_grid, 
#                       scoring='accuracy', 
#                       cv=cv,
#                       n_jobs=-1)
# search.fit(X_train, y_train)

In [ ]:
results_df = pd.DataFrame(search.cv_results_)
results_df = results_df.sort_values(by=["rank_test_score"])
results_df = results_df.set_index(
    results_df["params"].apply(lambda x: "_".join(str(val) for val in x.values()))
).rename_axis("kernel")
results_df[["params", "rank_test_score", "mean_test_score", "std_test_score"]]

In [ ]:
yhat_test = search.best_estimator_.predict(X_test)
print(y_test.shape)
print(accuracy_score(yhat_test, y_test))

In [ ]:
len(df[df[target] == 1])/len(df)

In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, yhat_test)

# Print the confusion matrix
print("Confusion Matrix:")
print(cm)

In [ ]:
yhat_train = search.best_estimator_.predict(X_train)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

# Calculate the confusion matrix
cm = confusion_matrix(y_train, yhat_train)

# Create a heatmap using seaborn
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues",
            xticklabels=['yes', 'no'], yticklabels=['yes', 'no'])
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()


In [ ]:
# I am sure that more estimators is generating better results.

rfmodel = RandomForestClassifier(
    n_estimators=500,
    max_depth=50,
    min_samples_split=50,
    n_jobs=-1,
    verbose=1
).fit(df_encoded[features], df_encoded[target])

In [ ]:
# Calculate the confusion matrix
yhat_rf = rfmodel.predict(df_encoded[features])
cm = confusion_matrix(df_encoded[target], yhat_rf)

# Create a heatmap using seaborn
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues",
            xticklabels=['yes', 'no'], yticklabels=['yes', 'no'])
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()


In [ ]:
from xgboost import XGBClassifier

xgb_model = XGBClassifier()

xgb_model.fit(df_encoded[features], df_encoded[target])